In [1]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [3]:
# spark master
spark_master = "local[*]"

spark = SparkSession.builder\
            .master(spark_master) \
            .appName("HW 2025")\
            .getOrCreate()

In [ ]:
#Q1 Spark version
spark

In [5]:
#File location https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
file_path = os.getcwd()+"/data/Raw/yellow_tripdata_2024-10.parquet"

folder_path = os.getcwd()+"/data/Processed/Yellow/"

In [6]:
# Q2 Repartition the Dataframe to 4 partitions and save it to parquet.
spark.read.parquet(file_path).repartition(4)\
    .write.parquet(folder_path)

In [7]:
#Get file size
os.path.getsize(folder_path+[f for f in os.listdir(folder_path) if f.endswith(".parquet")][0])/ (1024 * 1024)

22.386794090270996

In [22]:
#read parquet df

df = spark.read.parquet(folder_path)

In [23]:
df.limit(10).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-10-07 16:40:43|  2024-10-07 18:10:56|              1|         14.8|        99|                 N|         127|         225|           1|       47.5|  0.0|    0.5|       0.

In [16]:
#How many taxi trips were there that started on the 15th of October?
df.filter(f.to_date(f.col('tpep_pickup_datetime')) == '2024-10-15')\
    .select(f.count('*').alias('count_rows')).show()

+----------+
|count_rows|
+----------+
|    128893|
+----------+



In [20]:
# What is the length of the longest trip in the dataset in hours?

df.withColumn('Trip_length', f.to_timestamp(f.col('tpep_dropoff_datetime')).cast('long')-f.to_timestamp(f.col('tpep_pickup_datetime')).cast('long'))\
.select((f.max('Trip_length')/3600).alias('longest_trip')).show()

+------------------+
|      longest_trip|
+------------------+
|162.61777777777777|
+------------------+



In [21]:
# what is the name of the LEAST frequent pickup location Zone?
df_zones = spark.read.format('csv')\
        .option('header','true')\
        .load(os.getcwd()+"/data/taxi_zone_lookup.csv")\
        .select(f.col('LocationID').alias('PUlocationID'),'Zone')

df.join(df_zones,'PUlocationID','inner')\
    .groupBy('Zone')\
    .agg(f.count('*').alias('cnt'))\
    .orderBy(f.col('cnt'))\
    .select('Zone').limit(1).show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
+--------------------+



In [ ]:
## if spark is local
# spark.stop()